# [문제]
- law_2.docx 파일을 읽고, Chroma에 저장
- LLM 질문 -> 답변
- 전세사기피해에 관한 법률 질문만 받기
- 이 외의 질문은 '답변을 할 수 없습니다.'

In [ ]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader = Docx2txtLoader('law_2.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

database = Chroma.from_documents(
    documents=document_list,
    collection_name='chroma-law-2',
    persist_directory='./chroma',
    embedding=embedding,
)

In [ ]:

database = Chroma(
    collection_name='chroma-law-2',
    persist_directory='./chroma',
    embedding_function=embedding,
)

query = '오늘의 점심 메뉴는?'

retrieved_docs = database.similarity_search(query=query, k=2)

prompt = '''
[identity]
- 당신은 전세사기피해 법률 전문가입니다.
- [context]를 참고하여 사용자의 질문에 답변해주세요.
- 전세사기피해 법률 이외의 질문에는 '답변을 할 수 없습니다.'로 답하세요.

[context]
{retrieved_docs}

Question: {query}
'''

context = '😊\n'.join([doc.page_content for doc in retrieved_docs]) # -> list 내 for 문

formatted_prompt = prompt.format(
    retrieved_docs=context, 
    query=query)

llm = ChatOpenAI()

ai_message = llm.invoke(formatted_prompt)
print(ai_message.content)

답변을 할 수 없습니다.
